In [1]:
import pandas as pd
import numpy as np

In [2]:
# Read in the data from a JSON file
df = pd.read_json('indeed-data-jobs-FINAL.json')
df.sample(5)

,job_title,company,location,is_remote,salary,post_date,date_accessed,job_description
808,Machine Learning Engineer,DeepHealth,"Cambridge, MA",0,,30+ days ago,2021-03-05,Description DeepHealth is a digital health sta...
536,Data Engineer,Morris Animal Foundation,"Denver, CO 80246",1,"$61,900 - $77,300 a year",2 days ago,2021-03-05,Position Title: Data EngineerDepartment: Inf...
431,Data Analyst,Aiteq,Remote,1,,30+ days ago,2021-03-05,your dutiespráce v mezinárodním týmuznalost pr...
426,Tableau Developer/Data Analyst,Peak Mountain Technology Solutions,Remote,1,,17 days ago,2021-03-05,Tableau BI Developer/AnalystDescription: The r...
508,Data Engineer,Expedite Technologies Solutions,"Atlanta, GA 30301",1,$50 - $60 an hour,2 days ago,2021-03-05,"Job Title: Data Engineer Location: Austin, TXD..."


In [3]:
# these were probably removed so I'll remove them from the data
df[df.job_description == '']

,job_title,company,location,is_remote,salary,post_date,date_accessed,job_description
303,,,,0,,,2021-03-05,
463,,,,0,,,2021-03-05,
586,,,,0,,,2021-03-05,
755,,,,0,,,2021-03-05,


In [4]:
# REMOVED ROWS ABOVE
df = df[df.job_description != '']

In [5]:
# rows, columns
df.shape

(898, 8)

In [6]:
# data summary
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 898 entries, 0 to 901
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   job_title        898 non-null    object
 1   company          898 non-null    object
 2   location         898 non-null    object
 3   is_remote        898 non-null    int64 
 4   salary           898 non-null    object
 5   post_date        898 non-null    object
 6   date_accessed    898 non-null    object
 7   job_description  898 non-null    object
dtypes: int64(1), object(7)
memory usage: 63.1+ KB


# Creating Labels
### I'll need to create a script that uses the job title to create the label
I already see that data entry isn't what I'm looking for so I'll need to iterate over all entries to purge jobs like this that don't fall into 1 of my 4 categories

In [7]:
df.job_title

0                                      Data Entry
1                                  Data Scientist
2                                  Data Scientist
3                                  Data Scientist
4      Early Career Data Scientist - Applied Math
                          ...                    
897      Machine Learning Engineer, Ads Relevance
898                 Machine Learning: AI Engineer
899                     Machine Learning Engineer
900             Applied Machine Learning Engineer
901                     Machine Learning Engineer
Name: job_title, Length: 898, dtype: object

In [8]:
pd.DataFrame(df.loc[0]).T

,job_title,company,location,is_remote,salary,post_date,date_accessed,job_description
0,Data Entry,TransPerfect Translations Inc.,Remote,1,,6 days ago,2021-03-05,We are looking for Data Entry Clerks to join o...


In [9]:
### Regex

In [10]:
df

,job_title,company,location,is_remote,salary,post_date,date_accessed,job_description
0,Data Entry,TransPerfect Translations Inc.,Remote,1,,6 days ago,2021-03-05,We are looking for Data Entry Clerks to join o...
1,Data Scientist,ForMotiv,Remote,1,"$75,000 - $120,000 a year",30+ days ago,2021-03-05,Has it ever occurred to you that as the Intern...
2,Data Scientist,Redzara.com,Remote,1,$35 - $80 an hour,10 days ago,2021-03-05,Only GC / EAD only. No C2CBackground screening...
3,Data Scientist,Nova Collective,Remote,1,$35 - $48 an hour,24 days ago,2021-03-05,Are you a data scientist who is really excited...
4,Early Career Data Scientist - Applied Math,Pacific Northwest National Laboratory,"Seattle, WA",0,,1 day ago,2021-03-05,Organization and Job ID Job ID: 311747 Directo...
...,...,...,...,...,...,...,...,...
897,"Machine Learning Engineer, Ads Relevance",Pinterest,"Palo Alto, CA",0,,24 days ago,2021-03-05,About Pinterest: Millions of people across th...
898,Machine Learning: AI Engineer,"ALTRON, Inc.","Fairfax, VA",0,,30+ days ago,2021-03-05,Altron Inc. is seeking a Machine Learning (ML)...
899,Machine Learning Engineer,Global Payments,"Alpharetta, GA 30022",0,,24 days ago,2021-03-05,"Every day, Global Payments makes it possible f..."
900,Applied Machine Learning Engineer,Anomaly,"New York, NY 10012",0,,30+ days ago,2021-03-05,About Anomaly Anomaly is a healthcare technol...


## [FAIL] Finding each Label by Indexing
### The data is sort of in order by the keyword I used while scraping

- 1st: 'data scientist'
- 2nd: 'data analyst'
- 3rd: 'data engineer'
- 4th: 'machine learning engineer'

*I scraped about the same amount of jobs for each of the 4 labels. So I'm going to index the dataframe for the range within each quartile because I know that the job titles will change around that point so I'll just differentiate the labels by peeking into the DataFrame from the reference points.*

In [11]:
print(f'1st quarter: {898*.25} \n 2nd quarter: {898*.5} \n 3rd quarter: {898*.75}')

1st quarter: 224.5 
 2nd quarter: 449.0 
 3rd quarter: 673.5


In [12]:
# the last data scientist entry was at 223
#slice1 = slice(0,224)
#(df[:224] == df[slice1]).sum()

In [13]:
# the last data analyst entry was at 447
# df[224:447]

In [14]:
# the last data engineer entry was at 676
# df[447:674]

In [15]:
# the rest of the jobs are machine learning engineers
#df[674:]

In [16]:
def create_labels(df, first=slice(0,224), second=slice(224,447), third=slice(447,674), last=slice(674,902)):
    '''
    This function takes in a DataFrame and creates labels based on
    the index given as as second parameter. Note: this assumes your
    labels have been presorted and you know the index ranges for each.
    ''' 
    df['label'] = np.where(df.iloc[first], 'Data Scientist', '')
    df['label'] = np.where(df.iloc[second], 'Data Analyst', '')
    df['label'] = np.where(df.iloc[third], 'Data Engineer', '')
    df['label'] = np.where(df.iloc[last], 'Machine Learning Engineer', '')
    return df

In [17]:
df.sample()

,job_title,company,location,is_remote,salary,post_date,date_accessed,job_description
628,Data Engineer (SQL),Piper Companies,"Conshohocken, PA",0,$60 - $70 an hour,3 days ago,2021-03-05,Piper Companies is looking for a SQL Data Engi...


In [18]:
#df.insert(8, 'label', 'Data Scientist', allow_duplicates=True)

### FAIL ^ Trying regex

In [19]:
# test case 1
import re

text = 'Early Career Data Scientist - Applied Math data scientist'

r=r'.*?(data).*?(scientist).*?'

m = re.search(r, text, flags=re.IGNORECASE)
m

<re.Match object; span=(0, 27), match='Early Career Data Scientist'>

In [20]:
m[0], m[1], m[2]

('Early Career Data Scientist', 'Data', 'Scientist')

In [21]:
# test case 2
text = 'Entry-Level Business Analyst'

r=r'.*?(business).*?(analyst).*?'

m = re.search(r, text, flags=re.IGNORECASE)
m

<re.Match object; span=(0, 28), match='Entry-Level Business Analyst'>

In [22]:
m[0], m[1], m[2]

('Entry-Level Business Analyst', 'Business', 'Analyst')

In [23]:
# test case 3
text = 'Machine Learning Application Engineer in Healt...'

r=r'.*?(machine).*?(learning).*?(engineer).*?'

m = re.search(r, text, flags=re.IGNORECASE)
m

<re.Match object; span=(0, 37), match='Machine Learning Application Engineer'>

In [24]:
m[0], m[1], m[2], m[3]

('Machine Learning Application Engineer', 'Machine', 'Learning', 'Engineer')

In [25]:
def create_labels(df):
    for job in df.job_title:
        if 'machine learning engineer' in job.lower():
            df.assign(label = 'MLE')
        elif 'data analyst' in job.lower() or 'business analyst' in job.lower():
            df.assign(label = 'DA')
        elif 'data scientist' in job.lower():
            df.assign(label = 'DS')
        elif 'data engineer' in job.lower():
            df.assign(label = 'DE')
        return df

In [26]:
def create_labels(df):
    df['label'] = ' '
    for job in df.job_title:
        df['label'] = np.where(
            'machine learning engineer' in job.lower(),
            'MLE',
            df['label'])
        df['label'] = np.where(
            'data analyst' in job.lower() or 'business analyst' in job.lower(),
            'DA',
            df['label'])
        df['label'] = np.where(
            'data scientist' in job.lower(),
            'DS',
            df['label'])
        df['label'] = np.where(
            'data engineer' in job.lower(),
            'DE',
            df['label'])
        return df

In [27]:
df['label'] = ' '
for job in df.job_title:
    df['label'] = np.where(
                'machine learning engineer' in job.lower() == True,
                'MLE',
                '')
    df['label'] = np.where(
                'data analyst' in job.lower() or 'business analyst' in job.lower() == True,
                'DA',
                '')    
    df['label'] = np.where(
                'data scientist' in job.lower(),
                'DS',
                '')
    df['label'] = np.where(
                'data engineer' in job.lower(),
                'DE',
                '')

In [56]:
# 58 nulls
df['label'] = df.job_title.str.lower().str.extract('(.*?data.*?scientist.*?|.*?data.*?analyst.*?|data.*?engineer.*?|.*?machine.*?learning.*?engineer.*?)')

In [56]:
df['label'] = df.job_title.str.lower().str.extract('(.*?data.*?scientist.*?|.*?data.*?analyst.*?|data.*?engineer.*?|.*?machine.*?learning.*?engineer.*?)')

In [63]:
df[['job_title', 'label']].sample(5)

,job_title,label
859,Machine Learning Engineer,machine learning engineer
318,Online Data Analyst,online data analyst
778,"Machine Learning Engineer, Cameos",machine learning engineer
474,Data Engineer,data engineer
248,Data Analyst -Enterprise Business Systems (1.0...,data analyst


In [71]:
df[df.label.isnull()]

,job_title,company,location,is_remote,salary,post_date,date_accessed,job_description,label
0,Data Entry,TransPerfect Translations Inc.,Remote,1,,6 days ago,2021-03-05,We are looking for Data Entry Clerks to join o...,NaN
23,"Associate Scientist, Data Support",Pfizer,"Groton, CT",0,,2 days ago,2021-03-05,"Pfizer colleagues have many perspectives, many...",NaN
33,IT Data Analytics Intern,Farmers Insurance Group,Remote,1,$20.50 an hour,24 days ago,2021-03-05,IT Intern: IT Data Analytics intern We’re loo...,NaN
194,Deep Learning Scientist,SimBioSys,"Champaign, IL",0,,11 days ago,2021-03-05,SimBioSys (http://www.simbiosys.com) is seekin...,NaN
246,"Analyst, Data and Analysis",Digitas,"New York, NY 10104",0,,1 day ago,2021-03-05,"What you’ll do: As an Analyst, you’ll lay the ...",NaN
260,"Analyst, Data and Analysis",Digitas,"New York, NY 10104",0,,1 day ago,2021-03-05,"What you’ll do: As an Analyst, you’ll lay the ...",NaN
264,Data Specialist,Roman Catholic Bishop of Salt Lake City,"Salt Lake City, UT 84103",0,"$39,000 - $45,000 a year",1 day ago,2021-03-05,DATA SPECIALISTOFFICE OF STEWARDSHIP & DEVELOP...,NaN
319,"Analyst, Data & Research",Los Angeles Homeless Services Authority,"Los Angeles, CA 90017",0,"$52,854 - $76,638 a year",1 day ago,2021-03-05,The Los Angeles Homeless Services Authority (L...,NaN
344,Data Specialist,Wunderman Thompson,"Louisville, CO",0,,1 day ago,2021-03-05,Who we are: At Wunderman Thompson we exist to...,NaN
349,Security Analyst,Abnormal Security,"Lehi, UT",0,"$30,000 a year",30+ days ago,2021-03-05,About Abnormal Security: Abnormal Security is...,NaN


In [70]:
len((df[df.label.isnull()]))

58

In [49]:
re.search(r'.*?(data).*(scientist).*?', 'Data Science Engineer/Scientist'.lower()).group()

'data science engineer/scientist'